In [ ]:
### Setup the installation 
conda create -n project pip python=3.8
conda activate project

In [ ]:
# Install the TensorFlow PIP package
pip install --ignore-installed --upgrade tensorflow==2.2.0

In [ ]:
# Install dependency packages
conda install -c anaconda cudatoolkit=10.1
conda install -c anaconda protobuf=3.13.0
conda install cython=0.29.21

In [ ]:
# Verify your Installation
python -c "import tensorflow as tf;print(tf.reduce_sum(tf.random.normal([1000, 1000])))"

In [ ]:
#it is time to install the TensorFlow Object Detection API.
mkdir TensorFlow
cd TensorFlow

In [ ]:
# clone the TensorFlow Models repository
git clone https://github.com/tensorflow/models.git

In [ ]:
# Protobuf Compilation
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
cd 

In [ ]:
# COCO API installation - a dependency of the Object Detection API.
git clone https://github.com/cocodataset/cocoapi.git
cd cocoapi/PythonAPI
make
cp -r pycocotools TensorFlow/models/research/

In [ ]:
# From within TensorFlow/models/research/
cd TensorFlow/models/research/
cp object_detection/packages/tf2/setup.py .
python -m pip install . # dont worry with the issues

In [ ]:
# Test your Installation
# From within TensorFlow/models/research/
python object_detection/builders/model_builder_tf2_test.py

In [ ]:
# Preparing the Workspace
cd 
cd TensorFlow
mkdir workspace
cd workspace
mkdir training_demo
cd training_demo

In [ ]:
# structure for training folders
mkdir annotations # This folder will be used to store all *.csv files and *.record files, 
mkdir exported-models #  used to store exported versions of our trained model(s).
mkdir images # contains a copy of all the images in our dataset
mkdir models # contain a sub-folder for each of training job
mkdir pre-trained-models # will contain the downloaded pre-trained models

In [ ]:
cd images

In [ ]:
# Copy both image and xml

In [ ]:
mkdir train
cd train
cp -vr /home/Data/Cognitive_Data/training_final/* .
cd ..
mkdir test
cd test
cp -vr /home/Data/Cognitive_Data/validation_final/* .

In [ ]:
cd ../../

In [ ]:
mkdir scripts
cd scripts
mkdir preprocessing
cd preprocessing

In [ ]:
# Create Label Map
cd ../../
cd training_demo/annotations
nano label_map.pbtxt

In [ ]:
# replace label1,2,3 with label names

In [ ]:
# item {
#     id: 1
#     name: 'label1'
# }

# item {
#     id: 2
#     name: 'label2'
# }
# item {
#     id: 3
#     name: 'label3'
# }

In [ ]:
# Create TensorFlow Records
cd TensorFlow/scripts/preprocessing
wget https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/9da745a96c2281fec59c2035d0b50d0b/partition_dataset.py

In [ ]:
python generate_tfrecord.py -x C:/Users/Documents/Tensorflow/workspace/training_demo/images/train -l C:/Users/Documents/Tensorflow/workspace/training_demo/annotations/label_map.pbtxt -o C:/Users/Documents/Tensorflow/workspace/training_demo/annotations/train.record
python generate_tfrecord.py -x C:/Users/Documents/Tensorflow/workspace/training_demo/images/test -l C:/Users/Documents/Tensorflow2/workspace/training_demo/annotations/label_map.pbtxt -o C:/Users/Documents/Tensorflow/workspace/training_demo/annotations/test.record

In [ ]:
# Download Pre-Trained Model
cd training_demo/pre-trained-models
wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8.tar.gz

In [ ]:
tar -xvf ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8.tar.gz 

In [ ]:
# Configure the Training Pipeline
cd training_demo/models
mkdir ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8
cd ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8

In [ ]:
cp training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config .

In [ ]:
# look at the changes that we shall need to apply to the pipeline.config file
 num_classes: 3
fine_tune_checkpoint_type: "detection"
TensorFlow/workspace/training_demo/pre-trained-models/ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8/checkpoint/ckpt-0
TensorFlow/workspace/training_demo/annotations/label_map.pbtxt
TensorFlow/workspace/training_demo/annotations/train.record
TensorFlow/workspace/training_demo/annotations/test.record
batch_size=1 # if it not run

In [ ]:
# Training the Model
cp TensorFlow/models/research/object_detection/model_main_tf2.py training_demo
cp models/research/object_detection/model_main_tf2.py workspace/training_demo/

In [ ]:
export CUDA_VISIBLE_DEVICES=3
## checking
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]

In [ ]:
conda install -c anaconda cudnn=7.6.5

In [ ]:
python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config
python model_main_tf2.py --model_dir=models/ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8/ --pipeline_config_path=models/ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8/pipeline.config

In [ ]:
# Note:
# it seems that it is advisable to allow you model to reach a TotalLoss of at least 2 (ideally 1 and lower) if you want to achieve “fair” detection results. 

In [ ]:
export CUDA_VISIBLE_DEVICES=2
## checking
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]

In [ ]:
# Evaluating the Model 
python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config --checkpoint_dir=models/my_ssd_resnet50_v1_fpn

In [ ]:
# Monitor Training Job Progress using TensorBoard
cd training_demo
tensorboard --logdir=models/my_ssd_resnet50_v1_fpn

In [ ]:
# Exporting a Trained Model
cp TensorFlow/models/research/object_detection/exporter_main_v2.py training_demo

In [ ]:
cd training_demo
python exporter_main_v2.py --input_type image_tensor --pipeline_config_path models/my_ssd_resnet50_v1_fpn/pipeline.config --trained_checkpoint_dir models/my_ssd_resnet50_v1_fpn/ --output_directory exported-models\my_ssd_resnet50_v1_fpn